In [1]:
import pandas as pd
import openai
import csv
import time
import os
import regex as re
import math
import random

import openai

pd.set_option('display.max_colwidth', None)
file_path = "D:/api_key.txt"
with open(file_path, 'r') as file:
    api_key = file.read()

openai.api_key = (api_key)

In [2]:
voorbeeld_zin = 'meneer is omgevallen'
voorbeeld_zin2 = 'mevrouw heeft goed gegeten'
generate_faulty = "Genereer medische tekst met grammaticale en spellingsfouten. De tekst beschrijft dagelijkse gebeurtenissen van de client. Maak zeer diverse tekst. Bijvoorbeeld: " + voorbeeld_zin + "Bijvoorbeeld: " + voorbeeld_zin2
message_history = []

message_history.append({'role' : "system", "content": generate_faulty})
#message_history.append({"role": "user", 'content' : paraphrase_input})

completion = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',messages=message_history,
    temperature=1,
    top_p=0.9
)
#write header to file. 
#write original sentence to file and correction.
reply_content = completion.choices[0].message.content
print(reply_content)

Vandag heef mevrrouw Van Dijk een gowd nacht rus gehaat. Ze werd om 8 uur wakker met een klain beetje hoofdpijn. Mevrrouw Van Dijk besloot om een aspirine te nemen om de pijn te verlichten. Na het nemen van de aspirine voelde ze zich een beetje beter. Ze stond op en liep naar de keuken om ontbiyt te maken.

Mevrrouw Van Dijk heef besloten om een eitsandwich te maken voor ontbiyt. Ze klukte de eieren in een koekepan en zette ze op het fornuis. Terwijl ze wachtte tot de eieren gekookt waren, sneed ze een tomaat en een schijfje kaas. Toen de eieren klaar waren, legde ze ze op een sneetje brood en voegde ze de tomaat en kaas toe. Mevrrouw Van Dijk at langzaam haar ontbiyt op en genoot van elke hap.

Na het ontbiyt besloot mevrrouw Van Dijk om een wandeling te maken in het parwk. Ze trok haar jas aan en ging naar buiten. Het was een koude dag, maar de frisse lucht deed haar goed. Ze wandelde rustig door het parwk en genoot van de natuur om haar heen.

In de middag had mevrrouw Van Dijk een 

In [3]:
def genereer_data(voorbeeld_zin, voorbeeld_zin2,temp=1,top_p=0.9):
    generate_faulty = "Genereer medische tekst met grammaticale en spellingsfouten. De tekst beschrijft dagelijkse gebeurtenissen van de client onder zorg. Maak zeer diverse tekst. Bijvoorbeeld: " + voorbeeld_zin + "Bijvoorbeeld: " + voorbeeld_zin2
    message_history = []
    message_history.append({'role' : "system", "content": generate_faulty})
    message_history.append({'role' : "system", "content" : 'genereer 1 tekst'})
    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',messages=message_history,
        temperature=temp,
        top_p=top_p
    )
    #write header to file. 
    #write original sentence to file and correction.
    reply_content = completion.choices[0].message.content
    return reply_content

In [3]:
def sample_from_df(df):
    random_row = df.sample()
    # extract the values of the sampled row
    values = random_row.values[0]
    # print the values as a string
    voorbeeld_zin = (''.join(map(str, values)))

    same = True
    while same == True:
        random_row = df.sample()
        # extract the values of the sampled row
        values = random_row.values[0]
        # print the values as a string
        voorbeeld_zin2= (''.join(map(str, values)))
        if voorbeeld_zin != voorbeeld_zin2:
            same = False

    

    return voorbeeld_zin, voorbeeld_zin2

In [4]:
def generate_faulty_data(df, output_file_path, amount_per_combination, temp, top_p, generation_limit):
    # check if output file already exists
    if os.path.exists(output_file_path):
        # if it does, read the last row that was successfully processed
        with open(output_file_path, 'r', encoding='utf-8') as output_file:
            reader = csv.reader(output_file)
            for row in reader:
                pass
            last_row_processed = reader.line_num
    else:
        # if it doesn't, set last_row_processed to 0
        last_row_processed = 0

    # open input and output files
    #ISO-8859-1
    with open(output_file_path, 'a', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)

        #If initializing a new document, skip the header and rewrite the new format.
        if last_row_processed == 0:
            writer.writerow(['generated', 'example_1', 'example2'])
        number_generated = 0
        while generation_limit > number_generated:
            time.sleep(20)
            voorbeeld_zin, voorbeeld_zin2 = sample_from_df(df)
            #print(reader.line_num)
            for i in range(amount_per_combination):
                # call API to manipulate data
                try:
                    reply_content = genereer_data(voorbeeld_zin, voorbeeld_zin2, temp=temp, top_p=top_p )
                    print(reply_content)
                except:
                    # if API call fails, wait for 5 seconds and retry
                    print('failed... retrying in 5 seconds')
                    time.sleep(5)
                    #Longer prompt costs more.
                    reply_content = genereer_data(voorbeeld_zin, voorbeeld_zin2, temp=temp, top_p=top_p )

                
                writer.writerow([reply_content, voorbeeld_zin, voorbeeld_zin2])
                number_generated += 1
              
                # update last_row_processed to current row number
                #last_row_processed = reader.line_num

                # flush output file buffer to ensure data is saved
                output_file.flush()

                # print progress
                #print(f"Processed row {last_row_processed}")
                print('Rows Processed:', number_generated)
                # break out of loop if num_rows is reached
                if generation_limit is not None and number_generated >= generation_limit:
                    print('BREAK')
                    break
        

    # delete output file if it's empty
    #if os.stat(output_file_path).st_size == 0:
    #    os.remove(output_file_path)


In [7]:
df = pd.read_csv("hm_correct_medical.csv")
generate_faulty_data(df=df, output_file_path='generated_medical_v3.csv', amount_per_combination=1, temp=1, top_p=0.9, generation_limit=500)

failed... retrying in 5 seconds
Rows Processed: 1
De patiënt klaagd over pijn in de buik en vraagt om een pill. De verpleegkundige geeft hem de medicijnt die hij nodig had. Later heeft de patiënt geprobeert om te eten, maar hij heeft het eten uitgebraken en het laken bevuild. De verpleegkundige heeft de patiënt geholpen om schoon te maken en het bed op te maken. De patiënt heeft ook gevraagt om een glas water, en de verpleegkundige heeft hem dit gegeeven.
Rows Processed: 2
De patiënt kom vandaag voor een controole van de hartslag. De dokter gaat het elektrokardiogramme maken om de hartactiviteiten te meten. Na de test, vind de dokter geen afwijkingen in het hartslag. Maar, de patiënt klaag over pijn in de benne. De dokter beslis om een radiografie te maken om te zien of er een breuk is. De patiënt gaat morgen de resultaten krijg en de behandeling zal dan bepaalt worden.
Rows Processed: 3
Gisteren ik ga op bezoek bij de patiënt. Ze klaagt over dat ze had veel pijn in haar been. Ik check